In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os
import pandas as pd
import glob
import math


In [2]:
File_path = '/bettik/bouissob/ISOMIP+/'
Bet_path = '/bettik/bouissob/'

In [3]:
os.listdir(File_path)[0:2]

['Ocean4_COM_MITgcm_JPL.nc', 'Ocean1_TYP_FVCOM.nc']

In [4]:
Data_Name = '*COM_NEMO-CNRS.nc'
paths = glob.glob(File_path + Data_Name)

In [5]:
paths

['/bettik/bouissob/ISOMIP+/Ocean1_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/ISOMIP+/Ocean4_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/ISOMIP+/Ocean3_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/ISOMIP+/Ocean2_COM_NEMO-CNRS.nc']

Dans NN, var en x pour simul y<br>
x : T, S, x, y, z<br>
y : MeltRate<br>
1ere Étape : Preparer les données

In [6]:
def get_depth_temp_sali(df, Data, zs):
    ys, IceDraft = df.ny, df.iceDraft
    N_T, N_S = [], []
    ylast = None
    dataset = Data[['temperatureYZ', 'salinityYZ']].to_dataframe().dropna()
    Index_Y = dataset.index.get_level_values('ny')
    for ind, y in enumerate(ys):
        indz = np.abs(np.array(zs + IceDraft[ind])).argmin()
        if y != ylast:
            Slice = dataset.iloc[Index_Y == y].droplevel('ny')
        Z_N = abs(np.array(Slice.index - indz)).argmin() + Slice.index[0]
        New_T = dataset.temperatureYZ.loc[Z_N, y]
        New_S = dataset.salinityYZ.loc[Z_N, y]
        N_T.append(New_T)
        N_S.append(New_S)
    return np.array(N_T), np.array(N_S)

In [8]:
for path in paths:
    dfT = pd.DataFrame()
    Name = path.replace('.nc', '').replace(File_path, '')
    print(path)
    Data = xr.open_dataset(path)
    T = np.array(Data.nTime.as_numpy())
    Horiz_resol = 2 #km/index
    XstartYZ = int(500 / Horiz_resol)
    YstartXZ = int(40 / Horiz_resol)
    time = len(T)

    for t in range(time):
        if (t+1)%int(time/10) == 0:
            print('Starting {} / {}'.format(t+1, time))
        df = pd.DataFrame()
        df['meltRate'] = (Data['meltRate'][t].to_dataframe())
        df['thermalDriving'] = Data['thermalDriving'][t].to_dataframe()
        df['halineDriving'] = Data['halineDriving'][t].to_dataframe()
        if Data['iceDraft'].ndim == 2:
            df['iceDraft'] = Data['iceDraft'].to_dataframe()
        else:
            df['iceDraft'] = Data['iceDraft'][t].to_dataframe()
        df['date'] = t
        df.dropna(inplace = True)
        df.reset_index(inplace=True)
        T, S = get_depth_temp_sali(df, Data.sel(nTime = t),Data.z.to_dataframe())
        df['temperatureYZ'] = T
        df['salinityYZ'] = S
        dfT = pd.concat([dfT, df], ignore_index = True)
    dfT.dropna(inplace = True)
    dfT.reset_index(drop = True, inplace = True)
    dfT.rename(columns={'nx' :'x', 'ny': 'y'}, inplace=True)
    dfT['x'] = dfT['x'] * Horiz_resol
    dfT['y'] = dfT['y'] * Horiz_resol
    pd.DataFrame.to_csv(dfT, Bet_path + 'Data/data_{}.csv'.format(Name), index = False)

/bettik/bouissob/ISOMIP+/Ocean1_COM_NEMO-CNRS.nc
Starting 24 / 240
Starting 48 / 240
Starting 72 / 240
Starting 96 / 240
Starting 120 / 240
Starting 144 / 240
Starting 168 / 240
Starting 192 / 240
Starting 216 / 240
Starting 240 / 240
/bettik/bouissob/ISOMIP+/Ocean4_COM_NEMO-CNRS.nc
Starting 120 / 1200
Starting 240 / 1200
Starting 360 / 1200
Starting 480 / 1200
Starting 600 / 1200
Starting 720 / 1200
Starting 840 / 1200
Starting 960 / 1200
Starting 1080 / 1200
Starting 1200 / 1200
/bettik/bouissob/ISOMIP+/Ocean3_COM_NEMO-CNRS.nc
Starting 120 / 1200
Starting 240 / 1200
Starting 360 / 1200
Starting 480 / 1200
Starting 600 / 1200
Starting 720 / 1200
Starting 840 / 1200
Starting 960 / 1200
Starting 1080 / 1200
Starting 1200 / 1200
/bettik/bouissob/ISOMIP+/Ocean2_COM_NEMO-CNRS.nc
Starting 24 / 240
Starting 48 / 240
Starting 72 / 240
Starting 96 / 240
Starting 120 / 240
Starting 144 / 240
Starting 168 / 240
Starting 192 / 240
Starting 216 / 240
Starting 240 / 240
